# Backtest

In [ ]:
from system.agent import Agent
from system.env import Env

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2


config = {"capital" : 100, "interval" : "1d",
          "symbols" : ["BTC", "ETH", "SOL", "DASH", "MATIC",
                       "TWT", "GALA", "EGLD", "XMR"],
          "start" : "2023", "end" : "2023"
          }

env = Env(**config)

params_1 = {"agentId" : 1, "symbol" : "BTC", "allocation" : 100, "env" : env, "policy_name" : "MOMENTUM"}

In [ ]:
agent_1 = Agent(**params_1)
agent_1.update_policy_params(params = 3)
agent_1.run_episode()

In [ ]:
agent_1.get_report()

In [ ]:
env = Env(**config)
params_2 = {"symbol" : "BTC", "allocation" : 100, "env" : env, "policy_name":"TRIPLEMA"}

agent_2 = Agent(**params_2)
agent_2.update_policy_params(params = (7, 14, 21))
agent_2.run_episode()

In [ ]:
agent_2.get_report()

In [ ]:
env.journal.portfolio_data

# Interaction multi agent

In [1]:
from magent import MAgentThread2
from system.env import Env

from threading import Thread, Condition, Event, Barrier

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2


config = {"capital" : 100, "interval" : "1d",
          "symbols" : ["BTC", "ETH", "SOL", "DASH", "MATIC",
                       "TWT", "GALA", "EGLD", "XMR"],
          "start" : "2023", "end" : "2023"
          }

env = Env(**config)

In [2]:
def custom_action():
    print(" *** Barrier passee *** ")


condition = Condition()
barrier = Barrier(2, action=custom_action)

agent_msg = {}
master_msg = {}

params_1 = {"agentId" : 1, "symbol" : "BTC", "allocation" : 100, "env" : env, "policy_name" : "MOMENTUM",
            "condition" : condition, "barrier" : barrier, "agent_msg" : agent_msg, "master_msg" : master_msg}
params_2 = {"agentId" : 2, "symbol" : "BTC", "allocation" : 100, "env" : env, "policy_name":"TRIPLEMA",
            "condition" : condition, "barrier" : barrier, "agent_msg" : agent_msg, "master_msg" : master_msg}


agent_1 = MAgentThread2(**params_1)
agent_1.update_policy_params(params = 3)

agent_2 = MAgentThread2(**params_2)
agent_2.update_policy_params(params = (7, 14, 21))

In [3]:
agent_1.start()
agent_2.start()

agent_1.join()
agent_2.join()

1 - date : 2023-02-14 00:00:00, BTC
2 - date : 2023-02-14 00:00:00, BTC
 *** Barrier passee *** 
2 - date : 2023-02-15 00:00:00, BTC
1 - date : 2023-02-15 00:00:00, BTC
 *** Barrier passee *** 
2 - date : 2023-02-16 00:00:00, BTC
1 - date : 2023-02-16 00:00:00, BTC
 *** Barrier passee *** 
1 - date : 2023-02-17 00:00:00, BTC
2 - date : 2023-02-17 00:00:00, BTC
 *** Barrier passee *** 
2 - date : 2023-02-18 00:00:00, BTC
1 - date : 2023-02-18 00:00:00, BTC
 *** Barrier passee *** 
1 - date : 2023-02-19 00:00:00, BTC
2 - date : 2023-02-19 00:00:00, BTC
 *** Barrier passee *** 
2 - date : 2023-02-20 00:00:00, BTC
1 - date : 2023-02-20 00:00:00, BTC
 *** Barrier passee *** 
2 - date : 2023-02-21 00:00:00, BTC
1 - date : 2023-02-21 00:00:00, BTC
 *** Barrier passee *** 
2 - date : 2023-02-22 00:00:00, BTC
1 - date : 2023-02-22 00:00:00, BTC
 *** Barrier passee *** 
2 - date : 2023-02-23 00:00:00, BTC
1 - date : 2023-02-23 00:00:00, BTC
 *** Barrier passee *** 
2 - date : 2023-02-24 00:00:00

In [4]:
env.journal.portfolio_data

,agentId,date,risk_value,available_value,capital,symbol
0,1,2023-02-14,100.000000,0.000000,100.000000,BTC
1,2,2023-02-14,100.000000,-100.000000,0.000000,BTC
2,2,2023-02-15,109.568582,-100.000000,9.568582,BTC
3,1,2023-02-15,109.568582,-100.000000,9.568582,BTC
4,2,2023-02-16,105.936439,-100.000000,5.936439,BTC
...,...,...,...,...,...,...
509,1,2023-10-28,0.000000,0.000000,0.000000,BTC
510,1,2023-10-29,0.000000,151.939096,151.939096,BTC
511,2,2023-10-29,0.000000,151.939096,151.939096,BTC
512,1,2023-10-30,151.939096,0.000000,151.939096,BTC


In [6]:
agent_1.trades_data

{'BTC': {'all':             agentId     price  quantity  position   side  \
  date                                                       
  2023-02-14        1  22199.84  0.004505         1   LONG   
  2023-02-14        2  22199.84  0.004505         1   LONG   
  2023-02-15        2  24324.05  0.004505         1   LONG   
  2023-02-15        1  24324.05  0.004505         1   LONG   
  2023-02-16        2  23517.72  0.004505         1   LONG   
  ...             ...       ...       ...       ...    ...   
  2023-10-27        2  33892.02  0.000000        -1  SHORT   
  2023-10-27        1  33892.02  0.000000         0   None   
  2023-10-28        2  34121.25  0.000000        -1  SHORT   
  2023-10-28        1  34121.25 -0.004506        -1  SHORT   
  2023-10-29        1  34525.89  0.000000         0   None   
  
                                         state    in_value   out_value  \
  date                                                                   
  2023-02-14    ('Open', 'LON